# **Playground for UnfiedSKG**

## Overview

This is the demo version of UnifiedSKG for you to play with. For full function(full dataset load, full seq2seq construction, model training, full metrics evluation...), please check out our [UnfiedSKG framework](https://github.com/hkunlp/unifiedskg) and resource collection website [unifiedskg.com](https://unifiedskg.com).😃

Steps you need to play with UnifiedSKG

1.   Copy this colab to your account
2.   Choose the task
3.   Load in trained weight(finetuned/prefix-tuning, multi-task prefix-tuning...)
4.   Input your flattened seq and have fun!🤩

FAQ:


1. Available weights see [Huggingface Platform](https://huggingface.co/hkunlp).
2. Why prefix-module is big(100MB for T5-base and 300MB for T5-large)? Because we keep the re-paramized weight for potential research. See original [Prefix-tuning paper](https://arxiv.org/abs/2101.00190) for more detail information.
3. Do we need to add prefix on text(convert to SQL, give me the answer... etc.) when input it? We recommend you have this prompt words because we train our models with that although it only have littel influence on prediction(we are not so sure actually).


## Preparation

Connect to Google Drive and cd to root path (deleted)

Import essential packeges



In [1]:
import sys
import os
import time
import torch
import datasets
from transformers import (
    HfArgumentParser,
    set_seed,
    AutoTokenizer
)
from utils.configue import Configure
from utils.training_arguments import WrappedSeq2SeqTrainingArguments

from filelock import FileLock
import nltk
with FileLock(".lock") as lock:
    nltk.download("punkt", quiet=True)
    nltk.download("stopwords", quiet=True)

In [2]:
!which python

/Users/mac/miniconda3/envs/py3.7pytorch1.8new/bin/python


## Play

In [3]:
def play(txt, model, tokenizer):
    print("=====❓Request=====")
    print(txt)
    tokenized_txt = tokenizer([txt], max_length=1024, padding="max_length", truncation=True)
    pred = tokenizer.batch_decode(
      model.generate(
        torch.LongTensor(tokenized_txt.data['input_ids']),
        torch.LongTensor(tokenized_txt.data['attention_mask']),
        num_beams=1, 
        max_length=256
        ), 
      skip_special_tokens=True 
    ) # More details see utils/dataset.py and utils/trainer.py
    print("=====💡Answer=====")
    print(pred)

In [4]:
def play_pred(txt, model, tokenizer):
    tokenized_txt = tokenizer([txt], max_length=1024, padding="max_length", truncation=True)
    pred = tokenizer.batch_decode(
      model.generate(
        torch.LongTensor(tokenized_txt.data['input_ids']),
        torch.LongTensor(tokenized_txt.data['attention_mask']),
        num_beams=1, 
        max_length=256
        ), 
      skip_special_tokens=True 
    )
    return pred

### Spider(with cell value)

Set and load the args

In [143]:
# Set args here for runnning on notebook, we make them out here to make it more illustrative.
sys.argv = ['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', # This is the name of your .py launcher when you run this line of code.
            # belows are the parameters we set, take spider for example
            '--cfg', 'Salesforce/T5_large_prefix_spider_with_cell_value.cfg', 
            '--output_dir', './tmp']
parser = HfArgumentParser((WrappedSeq2SeqTrainingArguments,))
training_args, = parser.parse_args_into_dataclasses()
set_seed(training_args.seed)
args = Configure.Get(training_args.cfg)

Load tokenizer and model(21->1 multitasked prefix)

In [144]:
# model_path = 'hkunlp/from_all_T5_base_prefix_spider_with_cell_value2'
model_path = 'hkunlp/from_all_T5_large_prefix_spider_with_cell_value2'
# model_path = '/Users/mac/Desktop/syt/Deep-Learning/Repos/UnifiedSKG/output/server_runs/A-T5_base_prefix_spider_with_cell_value-asr_mixed/checkpoint-79500/'
# model_path = '/Users/mac/Desktop/syt/Deep-Learning/Repos/UnifiedSKG/output/server_runs/A-T5_base_prefix_spider_with_cell_value-rewritten_mixed/checkpoint-56500/'

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
from models.unified.prefixtuning import Model
model = Model(args)
model.load(model_path)

prefix-tuning sequence length is 10.


Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

play

In [145]:
struct_in = "| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id"
text_in = "what is the minimum, average, and maximum age of all singers from France?"
# seq_out = "SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'"

In [146]:
play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
what is the minimum, average, and maximum age of all singers from France?; structed knowledge: | concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id
=====💡Answer=====
['select min(age), avg(age), max(age) from singer where country = "France"']


### WikiTQ

Set and load the args

In [ ]:
# Set args here for runnning on notebook, we make them out here to make it more illustrative.
sys.argv = ['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', # This is the name of your .py launcher when you run this line of code.
            # belows are the parameters we set, take wikitq for example
            '--cfg', 'Salesforce/T5_base_prefix_wikitq.cfg', 
            '--output_dir', './tmp']
parser = HfArgumentParser((WrappedSeq2SeqTrainingArguments,))
training_args, = parser.parse_args_into_dataclasses()
set_seed(training_args.seed)
args = Configure.Get(training_args.cfg)

Load tokenizer and model(21->1 multitasked prefix)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("hkunlp/from_all_T5_base_prefix_wikitq2", use_fast=False)
from models.unified.prefixtuning import Model
model = Model(args)
model.load("hkunlp/from_all_T5_base_prefix_wikitq2")

Downloading:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

prefix-tuning sequence length is 10.


Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

play

In [ ]:
struct_in = "col : series # | season # | title | notes | original air date row 1 : 1 | 1 | \"the charity\" | alfie, dee dee, and melanie are supposed to be helping | october 15, 1994 row 2 : 2 | 1 | \"the practical joke war\" | alfie and goo unleash harsh practical jokes on dee dee | october 22, 1994 row 3 : 3 | 1 | \"the weekend aunt helen came\" | the boy's mother, jennifer, leaves for the weekend and she leaves | november 1, 1994 row 4 : 4 | 1 | \"robin hood play\" | alfie's school is performing the play robin hood and alfie is | november 9, 1994 row 5 : 5 | 1 | \"basketball tryouts\" | alfie tries out for the basketball team and doesn't make it | november 30, 1994 row 6 : 6 | 1 | \"where's the snake?\" | dee dee gets a snake, but he doesn't | december 6, 1994 row 7 : 7 | 1 | \"dee dee's girlfriend\" | a girl kisses dee dee in front of harry and | december 15, 1994 row 8 : 8 | 1 | \"dee dee's haircut\" | dee dee wants to get a hair cut by cool doctor money | december 20, 1994 row 9 : 9 | 1 | \"dee dee runs away\" | dee dee has been waiting to go to a monster truck show | december 28, 1994 row 10 : 10 | 1 | '\"donnell's birthday party\" | donnell is having a birthday party and brags about all the | january 5, 1995 row 11 : 11 | 1 | \"alfie's birthday party\" | goo and melanie pretend they are dating and they leave alfie out of | january 19, 1995 row 12 : 12 | 1 | \"candy sale\" | alfie and goo are selling candy to make money for some expensive jacket | january 26, 1995 row 13 : 13 | 1 | \"the big bully\" | dee dee gets beat up at school and his friends try to teach | february 2, 1995"
text_in = "alfie's birthday party aired on january 19. what was the airdate of the next episode?"
# seq_out = "january 26, 1995"

In [ ]:
play("{} ; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

### DART

Set and load the args

In [ ]:
# Set args here for runnning on notebook, we make them out here to make it more illustrative.
sys.argv = ['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', # This is the name of your .py launcher when you run this line of code.
            # belows are the parameters we set, take dart for example
            '--cfg', 'Salesforce/T5_base_prefix_dart.cfg', 
            '--output_dir', './tmp']
parser = HfArgumentParser((WrappedSeq2SeqTrainingArguments,))
training_args, = parser.parse_args_into_dataclasses()
set_seed(training_args.seed)
args = Configure.Get(training_args.cfg)

Load tokenizer and model(21->1 multitasked prefix)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("hkunlp/from_all_T5_base_prefix_dart2", use_fast=False)
from models.unified.prefixtuning import Model
model = Model(args)
model.load("hkunlp/from_all_T5_base_prefix_dart2")

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

prefix-tuning sequence length is 10.


Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

play

In [ ]:
struct_in = "Mars Hill College : joined : 1973 | Mars Hill College : location : Mars Hill, North Carolina"
text_in = ""
# seq_out = "A school from Mars Hill, North Carolina, joined in 1973."

In [ ]:
play("{} ; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


=====❓Request=====
 ; structed knowledge: Mars Hill College : joined : 1973 | Mars Hill College : location : Mars Hill, North Carolina
=====💡Answer=====
['Mars Hill College, located in Mars Hill, North Carolina, joined in 1973.']


### MultiWoZ2.1

Set and load the args

In [ ]:
# Set args here for runnning on notebook, we make them out here to make it more illustrative.
sys.argv = ['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', # This is the name of your .py launcher when you run this line of code.
            # belows are the parameters we set, take dart for example
            '--cfg', 'Salesforce/T5_base_prefix_multiwoz.cfg', 
            '--output_dir', './tmp']
parser = HfArgumentParser((WrappedSeq2SeqTrainingArguments,))
training_args, = parser.parse_args_into_dataclasses()
set_seed(training_args.seed)
args = Configure.Get(training_args.cfg)

Load tokenizer and model(21->1 multitasked prefix)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("hkunlp/from_all_T5_base_prefix_multiwoz2", use_fast=False)
from models.unified.prefixtuning import Model
model = Model(args)
model.load("hkunlp/from_all_T5_base_prefix_multiwoz2")

Downloading:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

prefix-tuning sequence length is 10.


Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

play

In [ ]:
struct_in = "hotel-pricerange: cheap, dontcare, expensive, moderate; hotel-type: guesthouse, hotel; hotel-parking: dontcare, free, no, yes; hotel-book day: friday, monday, saturday, sunday, thursday, tuesday, wednesday; hotel-book people: 1, 2, 3, 4, 5, 6, 7, 8; hotel-book stay: 1, 2, 3, 4, 5, 6, 7, 8; hotel-area: centre, dontcare, east, north, south, west; hotel-stars: 0, 1, 2, 3, 4, 5, dontcare; hotel-internet: dontcare, no, yes; hotel-name: none; train-destination: none; train-day: dontcare, friday, monday, saturday, sunday, thursday, tuesday, wednesday; train-departure: none; train-arriveby: none; train-book people: 0, 1, 10, 15, 2, 3, 4, 5, 6, 7, 8, 9; taxi-destination: none; taxi-departure: none; taxi-leaveat: none; train-leaveat: none; attraction-area: cambridge, centre, dontcare, east, north, south, west; restaurant-pricerange: cheap, dontcare, expensive, moderate; restaurant-area: centre, east, north, south, west; restaurant-food: none; attraction-name: none; restaurant-name: none; attraction-type: architecture, boat, church, cinema, college, concerthall, entertainment, hotspot, multiple sports, museum, nightclub, park, special, swimmingpool, theatre; restaurant-book day: friday, monday, saturday, sunday, thursday, tuesday, wednesday; restaurant-book people: 1, 2, 3, 4, 5, 6, 7, 8; restaurant-book time: none; taxi-arriveby: none; "
text_in = "that does not matter as long as it has free wifi and parking . || i can help you with that . what is your price range ? | i need to book a hotel in the east that has 4 stars . | none"
# seq_out = "hotel pricerange none, hotel type none, hotel parking yes, hotel book day none, hotel book people none, hotel book stay none, hotel area east, hotel stars 4, hotel internet yes, hotel name none, train destination none, train day none, train departure none, train arriveby none, train book people none, taxi destination none, taxi departure none, taxi leaveat none, train leaveat none, attraction area none, restaurant pricerange none, restaurant area none, restaurant food none, attraction name none, restaurant name none, attraction type none, restaurant book day none, restaurant book people none, restaurant book time none, taxi arriveby none"

In [ ]:
play("{} ; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


=====❓Request=====
that does not matter as long as it has free wifi and parking . || i can help you with that . what is your price range ? | i need to book a hotel in the east that has 4 stars . | none ; structed knowledge: hotel-pricerange: cheap, dontcare, expensive, moderate; hotel-type: guesthouse, hotel; hotel-parking: dontcare, free, no, yes; hotel-book day: friday, monday, saturday, sunday, thursday, tuesday, wednesday; hotel-book people: 1, 2, 3, 4, 5, 6, 7, 8; hotel-book stay: 1, 2, 3, 4, 5, 6, 7, 8; hotel-area: centre, dontcare, east, north, south, west; hotel-stars: 0, 1, 2, 3, 4, 5, dontcare; hotel-internet: dontcare, no, yes; hotel-name: none; train-destination: none; train-day: dontcare, friday, monday, saturday, sunday, thursday, tuesday, wednesday; train-departure: none; train-arriveby: none; train-book people: 0, 1, 10, 15, 2, 3, 4, 5, 6, 7, 8, 9; taxi-destination: none; taxi-departure: none; taxi-leaveat: none; train-leaveat: none; attraction-area: cambridge, centre, 

### TabFact

Set and load the args

In [ ]:
# Set args here for runnning on notebook, we make them out here to make it more illustrative.
sys.argv = ['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', # This is the name of your .py launcher when you run this line of code.
            # belows are the parameters we set, take sql2text for example
            '--cfg', 'Salesforce/T5_base_prefix_tab_fact.cfg', 
            '--output_dir', './tmp']
parser = HfArgumentParser((WrappedSeq2SeqTrainingArguments,))
training_args, = parser.parse_args_into_dataclasses()
set_seed(training_args.seed)
args = Configure.Get(training_args.cfg)

Load tokenizer and model(21->1 multitasked prefix)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("hkunlp/from_all_T5_base_prefix_tab_fact2", use_fast=False)
from models.unified.prefixtuning import Model
model = Model(args)
model.load("hkunlp/from_all_T5_base_prefix_tab_fact2")

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

prefix-tuning sequence length is 10.


Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

play

In [ ]:
struct_in = "col : round | clubs remaining | clubs involved | winners from previous round | new entries this round | leagues entering at this round row 1 : first round | 156 | 86 | none | 86 | tff third league & turkish regional amateur league row 2 : second round | 113 | 108 | 43 | 65 | s\u00fcper lig & tff first league row 3 : third round | 59 | 54 | 54 | none | none row 4 : fourth round | 32 | 32 | 27 | 5 | s\u00fcper lig row 5 : fifth round | 16 | 16 | 16 | none | none row 6 : group stage | 8 | 8 | 8 | none | none row 7 : semi - finals | 4 | 4 | 4 | none | none row 8 : final | 2 | 2 | 2 | none | none"
text_in = "during the third round of the turkish cup , there be no new entry during that stage"
# seq_out = "entailed",

In [ ]:
play("{} ; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


=====❓Request=====
during the third round of the turkish cup , there be no new entry during that stage ; structed knowledge: col : round | clubs remaining | clubs involved | winners from previous round | new entries this round | leagues entering at this round row 1 : first round | 156 | 86 | none | 86 | tff third league & turkish regional amateur league row 2 : second round | 113 | 108 | 43 | 65 | süper lig & tff first league row 3 : third round | 59 | 54 | 54 | none | none row 4 : fourth round | 32 | 32 | 27 | 5 | süper lig row 5 : fifth round | 16 | 16 | 16 | none | none row 6 : group stage | 8 | 8 | 8 | none | none row 7 : semi - finals | 4 | 4 | 4 | none | none row 8 : final | 2 | 2 | 2 | none | none
=====💡Answer=====
['entailed']


### SQL2Text

Set and load the args

In [ ]:
# Set args here for runnning on notebook, we make them out here to make it more illustrative.
sys.argv = ['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', # This is the name of your .py launcher when you run this line of code.
            # belows are the parameters we set, take sql2text for example
            '--cfg', 'Salesforce/T5_base_prefix_sql2text.cfg', 
            '--output_dir', './tmp']
parser = HfArgumentParser((WrappedSeq2SeqTrainingArguments,))
training_args, = parser.parse_args_into_dataclasses()
set_seed(training_args.seed)
args = Configure.Get(training_args.cfg)

Load tokenizer and model(21->1 multitasked prefix)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("hkunlp/from_all_T5_base_prefix_sql2text2", use_fast=False)
from models.unified.prefixtuning import Model
model = Model(args)
model.load("hkunlp/from_all_T5_base_prefix_sql2text2")

Downloading:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

prefix-tuning sequence length is 10.


Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

Play

In [ ]:
struct_in = ""
text_in = "SELECT name ,  country ,  age FROM singer ORDER BY age DESC"

In [ ]:
play("{} ; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


=====❓Request=====
SELECT name ,  country ,  age FROM singer ORDER BY age DESC ; structed knowledge: 
=====💡Answer=====
['What are the names, countries, and ages of all singers, ordered by age descending?']


# Full Evaluation

## Spider

In [68]:
import json
from copy import deepcopy

from seq2seq_construction import spider
from third_party.spider.preprocess.get_tables import dump_db_json_schema

import numpy as np
from tqdm.notebook import tqdm
import editdistance
from nltk.translate.bleu_score import corpus_bleu
from nltk.tokenize.treebank import TreebankWordDetokenizer

from SpeakQL.Allennlp_models.utils.spider import process_sql, evaluation
from SpeakQL.Allennlp_models.utils.misc_utils import EvaluateSQL, EvaluateSQL_full, \
    Postprocess_rewrite_seq, Postprocess_rewrite_seq_freeze_POS, Postprocess_rewrite_seq_modify_POS

### (Test)

In [10]:
test_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter+phonemes.json'
db_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/database'

with open(test_path, 'r') as f:
    test_dataset = json.load(f)
len(test_dataset)

547

In [13]:
schema_cache = dict()

In [17]:
# db_id = 'formula_1'
# _s = dump_db_json_schema(
#         db_path + "/" + db_id + "/" + db_id + ".sqlite", db_id)
# _s

In [25]:
sample = deepcopy(test_dataset[0][0])

db_id = sample["db_id"]
if db_id not in schema_cache:
    schema_cache[db_id] = dump_db_json_schema(
        db_path + "/" + db_id + "/" + db_id + ".sqlite", db_id)
schema = schema_cache[db_id]

ex = {
    "query": sample["query"],
    "question": sample["question"],
    "gold_question": sample["gold_question"],
    "db_id": db_id,
    "db_path": db_path,
    "db_table_names": schema["table_names_original"],
#     "db_column_names": [
#         {"table_id": table_id, "column_name": column_name}
#         for table_id, column_name in schema["column_names_original"]
#     ],
    "db_column_names": {
        "table_id": [table_id for table_id, column_name in schema["column_names_original"]],
        "column_name": [column_name for table_id, column_name in schema["column_names_original"]]
    },
    "db_column_types": schema["column_types"],
    "db_primary_keys": [{"column_id": column_id} for column_id in schema["primary_keys"]],
    "db_foreign_keys": [
        {"column_id": column_id, "other_column_id": other_column_id}
        for column_id, other_column_id in schema["foreign_keys"]
    ],
}

In [ ]:
ex

In [52]:
struct_in = spider.serialize_schema(
    question=ex["question"],
    db_path=ex["db_path"],
    db_id=ex["db_id"],
    db_column_names=ex["db_column_names"],
    db_table_names=ex["db_table_names"],
    schema_serialization_type="peteshaw",
    schema_serialization_randomized=False,
    schema_serialization_with_db_id=True,
    schema_serialization_with_db_content=True,
    normalize_query=True,
)
struct_in

' | pets_1 | student : stuid , lname , fname , age , sex , major , advisor , city_code | has_pet : stuid , petid | pets : petid , pettype , pet_age , weight'

### Functions

In [10]:
schema_cache = dict()

db_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/database'

def get_uskg_sample(sample):
    db_id = sample["db_id"]
    if db_id not in schema_cache:
        schema_cache[db_id] = dump_db_json_schema(
            db_path + "/" + db_id + "/" + db_id + ".sqlite", db_id)
    schema = schema_cache[db_id]

    return {
        "query": sample["query"],
        "question": sample["question"],
        "db_id": db_id,
        "db_path": db_path,
        "db_table_names": schema["table_names_original"],
        "db_column_names": {
            "table_id": [table_id for table_id, column_name in schema["column_names_original"]],
            "column_name": [column_name for table_id, column_name in schema["column_names_original"]]
        },
        "db_column_types": schema["column_types"],
        "db_primary_keys": [{"column_id": column_id} for column_id in schema["primary_keys"]],
        "db_foreign_keys": [
            {"column_id": column_id, "other_column_id": other_column_id}
            for column_id, other_column_id in schema["foreign_keys"]
        ],
    }

def get_uskg_struct_in(uskg_sample):
    return spider.serialize_schema(
        question=uskg_sample["question"],
        db_path=uskg_sample["db_path"],
        db_id=uskg_sample["db_id"],
        db_column_names=uskg_sample["db_column_names"],
        db_table_names=uskg_sample["db_table_names"],
        schema_serialization_type="peteshaw",
        schema_serialization_randomized=False,
        schema_serialization_with_db_id=True,
        schema_serialization_with_db_content=True,
        normalize_query=True,
    )

In [ ]:
sample = deepcopy(test_dataset[20][0])
uskg_sample = get_uskg_sample(sample)
struct_in = get_uskg_struct_in(uskg_sample)
struct_in

In [ ]:
# for i, s in enumerate(test_dataset[:20]):
#     uskg_sample = get_uskg_sample(s[0])
#     struct_in = get_uskg_struct_in(uskg_sample)
#     if '(' in struct_in:
#         print(i, struct_in)

In [64]:
text_in = spider.normalize(uskg_sample["question"])
text_in

'find the first name and age of students who have a dog but do not have a cat is a pet.'

In [68]:
play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
find the first name and age of students who have a dog but do not have a cat is a pet.; structed knowledge:  | pets_1 | student : stuid , lname , fname , age , sex , major , advisor , city_code | has_pet : stuid , petid | pets : petid , pettype ( cat , dog ) , pet_age , weight
=====💡Answer=====
['select distinct t1.fname, t1.age from student as t1 join has_pet as t2 on t1.stuid = t2.stuid join pets as t3 on t2.petid = t3.petid where t3.pettype = "dog" except select distinct fname, t1.age from student as t1 join has_pet as t2 on t1.stuid = t2.st']


### Full eval

In [26]:
def _Postprocess_rewrite_seq_wrapper(cand_dict, pred_dict):
    _tags = pred_dict['rewriter_tags']
    _rewrite_seq = pred_dict['rewrite_seq_prediction']
    _question_toks = cand_dict['question_toks']
    return Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)

In [75]:
def _detokenize(toks):
    detokenizer = TreebankWordDetokenizer()
    return detokenizer.detokenize(toks)

In [78]:
def Full_evaluate_ILM(eval_version,
                      rewriter_ILM_pred_path,
                      test_dataset_path,
                      orig_dev_path,
                      test_output_path=None,
                      result_output_path=None,
                      ILM_rewrite_func=_Postprocess_rewrite_seq_wrapper,
                      detokenize_func=_detokenize):
    
    '''
    eval_version: simply for printing results 
    
    Example paths:
    rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    
    ILM_rewrite_func: Callable, args: (_tags, _rewrite_seq, _question_toks)
    detokenize_func: Callable, args: (toks)
    '''
    
    VERSION = eval_version
    
    with open(rewriter_ILM_pred_path, 'r') as f:
        rewriter_ILM_preds = [json.loads(l) for l in f.readlines()]
    with open(test_dataset_path, 'r') as f:
        test_dataset = json.load(f)
    with open(orig_dev_path, 'r') as f:
        orig_dev_dataset = json.load(f)

    # len(rewriter_ILM_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

    ## Quick evaluation: only using the 1st ASR candidate

    pred_idx = 0

    ref_list = []
    hyp_list = []
    wer_numer = 0
    wer_denom = 0

    for d in tqdm(test_dataset, desc=f'VERSION {VERSION}'):
        if len(d) == 0:
            continue

        c = d[0]

        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

        # Debug 
        # assert c['rewriter_tags'] == p['rewriter_tags'][:len(c['rewriter_tags'])], f"{c['rewriter_tags']}\n{p['rewriter_tags']}\nShould raise"

        _db_id = o['db_id']

        _tags = p['rewriter_tags']
        _rewrite_seq = p['rewrite_seq_prediction']
        _question_toks = c['question_toks']

        # _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
        _rewritten_question_toks = ILM_rewrite_func(c, p)
        # _rewritten_question = ' '.join(_rewritten_question_toks)
        _rewritten_question = detokenize_func(_rewritten_question_toks)

        #_pred_sql = Question(_rewritten_question, _db_id, model_dict=model_dicts[rat_sql_run])[0]['inferred_code']
        
        _uskg_sample = get_uskg_sample(c)
        _uskg_sample['question'] = _rewritten_question
        _struct_in = get_uskg_struct_in(_uskg_sample)
        _pred_sql = play_pred("{}; structed knowledge: {}".format(_rewritten_question, _struct_in),
                              model, tokenizer)[0]

        _gold_sql = c['query']
        ## verbose=False since too many get_sql() errors...
        _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id, verbose=False)

        # Save the taggerILM raw outputs, for later aggregation 
        c['pred_tags'] = p['rewriter_tags']
        c['pred_ILM'] = p['rewrite_seq_prediction']
        c['pred_ILM_cands'] = p['rewrite_seq_prediction_cands']

        # Save prediction results 
        c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score
        c['exact'] = p['exact'] = _exact
        c['exec'] = p['exec'] = _exec

        _rewritten_question_toks = [_t.lower() for _t in _rewritten_question_toks]
        _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

        ref_list.append([_gold_question_toks])
        hyp_list.append(_rewritten_question_toks)
        wer_numer += editdistance.eval(_gold_question_toks, _rewritten_question_toks)
        wer_denom += len(_gold_question_toks)

        pred_idx += len(d)

    # Only using the 1st candidate to rewrite 
    _avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
    _avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
    _avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

    ## Std-dev (1st cand only)
    # _std_1st = np.std([d[0]['score'] for d in test_dataset])

    ## BLEU 
    _bleu = corpus_bleu(list_of_references=ref_list,
                        hypotheses=hyp_list)
    _wer = 1.0 * wer_numer / (wer_denom + 1e-9)

    print('='*20, f'VERSION: {VERSION}', '='*20)
    print('avg_exact = {:.4f}'.format(_avg_exact_1st))
    # print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
    print('avg = {:.4f}'.format(_avg_1st))
    print('avg_exec = {:.4f}'.format(_avg_exec_1st))
    print(f'BLEU = {_bleu:.4f}')
    print(f'WER = {_wer:.4f}')
    print('='*55)
    
    if test_output_path is not None:
        with open(test_output_path, 'w') as f:
            json.dump(test_dataset, f, indent=4)
    
    if result_output_path is not None:
        _res_d = {
            "avg_exact": _avg_exact_1st,
            "avg": _avg_1st,
            "avg_exec": _avg_exec_1st,
            "BLEU": _bleu,
            "WER": _wer,
        }
        with open(result_output_path, 'w') as f:
            json.dump(_res_d, f, indent=4)


In [80]:
VERSION_LIST = [f'2.12.1.{v}t-2.29.0.{v}i' for v in range(4)]
# VERSION_LIST += [f'2.12.1.{v}t-2.20.0.{v}i' for v in [0,1,2,3,4]]
# VERSION_LIST = [f'2.18.2.{v}i-oracle-tags' for v in range(5)]

HUMAN_TEST = False
ASR = 'Amazon'

for VERSION in VERSION_LIST:
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/output-{}.json'.format(VERSION)
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
        else:
            rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    else:
        rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

    # Non ASR
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/uskg-test-save/{VERSION}.json'
            result_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/uskg-test-save/eval-{VERSION}.json'
        else:
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-test-save/{VERSION}.json'
            result_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-test-save/eval-{VERSION}.json'
    else:
        test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-test-save/humantest-yshao-{VERSION}.json'
        result_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-test-save/eval-humantest-yshao-{VERSION}.json'
    # ASR
    # test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/{VERSION}.json'
    # Mixed
    # test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-mixed-test-save/{VERSION}.json'
    
    # temp
    test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-test-save/ATEMP-{VERSION}.json'
    result_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-test-save/ATEMP-eval-{VERSION}.json'
    
    Full_evaluate_ILM(eval_version=VERSION,
                      rewriter_ILM_pred_path=rewriter_ILM_pred_path,
                      test_dataset_path=test_dataset_path,
                      orig_dev_path=orig_dev_path,
                      test_output_path=test_output_path,
                      result_output_path=result_output_path)



VERSION 2.12.1.0t-2.29.0.0i:   0%|          | 0/547 [00:00<?, ?it/s]

--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['flights'], ['but']]
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

VERSION 2.12.1.1t-2.29.0.1i:   0%|          | 0/547 [00:00<?, ?it/s]

--- Not enough edits ---
Tags: ['O-KEEP', 'U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-DEL', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['all'], ['pg']]
==================== VERSION: 2.12.1.1t-2.29.0.1i ====================
avg_exact = 0.5027
avg = 0.6304
avg_exec = 0.5046
BLEU = 0.8832
WER = 0.0626


VERSION 2.12.1.2t-2.29.0.2i:   0%|          | 0/547 [00:00<?, ?it/s]

--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: []
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['ate'], ['but']]
--- Not enough edits ---
Tags: ['U-EDIT', 'U-DEL',

VERSION 2.12.1.3t-2.29.0.3i:   0%|          | 0/547 [00:00<?, ?it/s]

--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'U-EDIT', 'U-DEL', 'U-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['ids'], ['cv'], ["''"], ['?']]
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'U-EDIT', 'U-EDIT

#### rewritten-mixed 

In [126]:
model_path

'/Users/mac/Desktop/syt/Deep-Learning/Repos/UnifiedSKG/output/server_runs/A-T5_base_prefix_spider_with_cell_value-rewritten_mixed/checkpoint-56500/'

In [127]:
## Test of uskg-rewritten-mixed 

VERSION = '2.12.1.0t-2.29.0.0i'

os.makedirs('/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-rewritten-mixed-test-save', exist_ok=True)

rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-rewritten-mixed-test-save/{VERSION}.json'
result_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-rewritten-mixed-test-save/eval-{VERSION}.json'

Full_evaluate_ILM(eval_version=f'{VERSION} (rewritten-mixed)',
                  rewriter_ILM_pred_path=rewriter_ILM_pred_path,
                  test_dataset_path=test_dataset_path,
                  orig_dev_path=orig_dev_path,
                  test_output_path=test_output_path,
                  result_output_path=result_output_path)


VERSION 2.12.1.0t-2.29.0.0i (rewritten-mixed):   0%|          | 0/547 [00:00<?, ?it/s]

--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['flights'], ['but']]
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

### Blackbox eval

In [41]:
def Full_evaluate_Blackbox(eval_version,
                          test_dataset_path,
                          orig_dev_path,
                          test_output_path=None,
                          result_output_path=None):
    
    '''
    eval_version: simply for printing results 
    
    Example paths:
    rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    
    '''
    
    VERSION = eval_version
    
    with open(test_dataset_path, 'r') as f:
        test_dataset = json.load(f)
    with open(orig_dev_path, 'r') as f:
        orig_dev_dataset = json.load(f)

    # len(rewriter_ILM_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

    ## Quick evaluation: only using the 1st ASR candidate

    # pred_idx = 0

    ref_list = []
    hyp_list = []
    wer_numer = 0
    wer_denom = 0

    for d in tqdm(test_dataset, desc=f'VERSION {VERSION}'):
        if len(d) == 0:
            continue

        c = d[0]

        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

        # Debug 
        # assert c['rewriter_tags'] == p['rewriter_tags'][:len(c['rewriter_tags'])], f"{c['rewriter_tags']}\n{p['rewriter_tags']}\nShould raise"

        _db_id = o['db_id']
        #_pred_sql = Question(_rewritten_question, _db_id, model_dict=model_dicts[rat_sql_run])[0]['inferred_code']
        
        _uskg_sample = get_uskg_sample(c)
        # _uskg_sample['question'] = _rewritten_question
        _struct_in = get_uskg_struct_in(_uskg_sample)
        _pred_sql = play_pred("{}; structed knowledge: {}".format(_uskg_sample['question'], _struct_in),
                              model, tokenizer)[0]

        _gold_sql = c['query']
        ## verbose=False since too many get_sql() errors...
        _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id, verbose=False)

        # Save prediction results 
        c['pred_sql'] = _pred_sql
        c['score'] = _score
        c['exact'] = _exact
        c['exec'] = _exec

        _question_toks = c['question_toks']
        _question_toks = [_t.lower() for _t in _question_toks]
        _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

        ref_list.append([_gold_question_toks])
        hyp_list.append(_question_toks)
        wer_numer += editdistance.eval(_gold_question_toks, _question_toks)
        wer_denom += len(_gold_question_toks)

    # Only using the 1st candidate to rewrite 
    _avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
    _avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
    _avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

    ## Std-dev (1st cand only)
    # _std_1st = np.std([d[0]['score'] for d in test_dataset])

    ## BLEU 
    _bleu = corpus_bleu(list_of_references=ref_list,
                        hypotheses=hyp_list)
    _wer = 1.0 * wer_numer / (wer_denom + 1e-9)

    print('='*20, f'VERSION: {VERSION}', '='*20)
    print('avg_exact = {:.4f}'.format(_avg_exact_1st))
    # print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
    print('avg = {:.4f}'.format(_avg_1st))
    print('avg_exec = {:.4f}'.format(_avg_exec_1st))
    print(f'BLEU = {_bleu:.4f}')
    print(f'WER = {_wer:.4f}')
    print('='*55)
    
    if test_output_path is not None:
        with open(test_output_path, 'w') as f:
            json.dump(test_dataset, f, indent=4)
    
    if result_output_path is not None:
        _res_d = {
            "avg_exact": _avg_exact_1st,
            "avg": _avg_1st,
            "avg_exec": _avg_exec_1st,
            "BLEU": _bleu,
            "WER": _wer,
        }
        with open(result_output_path, 'w') as f:
            json.dump(_res_d, f, indent=4)


In [ ]:
HUMAN_TEST = False
ASR = 'Amazon'

if not HUMAN_TEST:
    if ASR == 'AssemblyAI':
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    else:
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
else:
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

# Non ASR
if not HUMAN_TEST:
    if ASR == 'AssemblyAI':
        test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/uskg-test-save/First-cands.json'
        result_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/uskg-test-save/eval-First-cands.json'

    else:
        test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-test-save/First-cands.json'
        result_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-test-save/eval-First-cands.json'
else:
    test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-test-save/humantest-yshao-First-cands.json'
    result_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-test-save/eval-humantest-yshao-First-cands.json'

Full_evaluate_Blackbox(eval_version='Blackbox (First-cand)',
                       test_dataset_path=test_dataset_path,
                       orig_dev_path=orig_dev_path,
                       test_output_path=test_output_path,
                       result_output_path=result_output_path)


#### orig human-test

In [149]:
model_path

'hkunlp/from_all_T5_large_prefix_spider_with_cell_value2'

In [150]:
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'
test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-large-test-save/humantest-yshao-First-cands.json'
result_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-large-test-save/eval-humantest-yshao-First-cands.json'

Full_evaluate_Blackbox(eval_version='Blackbox (First-cands) (Human-test)',
                       test_dataset_path=test_dataset_path,
                       orig_dev_path=orig_dev_path,
                       test_output_path=test_output_path,
                       result_output_path=result_output_path)


VERSION Blackbox (First-cands) (Human-test):   0%|          | 0/100 [00:00<?, ?it/s]

==================== VERSION: Blackbox (First-cands) (Human-test) ====================
avg_exact = 0.4500
avg = 0.6176
avg_exec = 0.4600
BLEU = 0.6934
WER = 0.1733


#### asr-mixed

In [116]:
model_path

'/Users/mac/Desktop/syt/Deep-Learning/Repos/UnifiedSKG/output/server_runs/A-T5_base_prefix_spider_with_cell_value-asr_mixed/checkpoint-79500/'

In [117]:
## Test of uskg-asr-mixed 

test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-mixed-test-save/First-cands.json'
result_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-mixed-test-save/eval-First-cands.json'

Full_evaluate_Blackbox(eval_version='Blackbox (First-cands)',
                       test_dataset_path=test_dataset_path,
                       orig_dev_path=orig_dev_path,
                       test_output_path=test_output_path,
                       result_output_path=result_output_path)


VERSION Blackbox (First-cands):   0%|          | 0/547 [00:00<?, ?it/s]

==================== VERSION: Blackbox (First-cands) ====================
avg_exact = 0.4954
avg = 0.6120
avg_exec = 0.4845
BLEU = 0.8010
WER = 0.1194


#### asr-mixed human-test

In [131]:
model_path

'/Users/mac/Desktop/syt/Deep-Learning/Repos/UnifiedSKG/output/server_runs/A-T5_base_prefix_spider_with_cell_value-asr_mixed/checkpoint-79500/'

In [132]:
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'
test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-mixed-test-save/humantest-yshao-First-cands.json'
result_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-mixed-test-save/eval-humantest-yshao-First-cands.json'


In [133]:
Full_evaluate_Blackbox(eval_version='Blackbox (First-cands) (Human-test)',
                       test_dataset_path=test_dataset_path,
                       orig_dev_path=orig_dev_path,
                       test_output_path=test_output_path,
                       result_output_path=result_output_path)

VERSION Blackbox (First-cands) (Human-test):   0%|          | 0/100 [00:00<?, ?it/s]

==================== VERSION: Blackbox (First-cands) (Human-test) ====================
avg_exact = 0.4600
avg = 0.6150
avg_exec = 0.4600
BLEU = 0.6934
WER = 0.1733


### Gold-text eval

In [38]:
def Full_evaluate_Gold_text(eval_version,
                          test_dataset_path,
                          orig_dev_path,
                          test_output_path=None,
                          result_output_path=None):
    
    '''
    eval_version: simply for printing results 
    
    Example paths:
    rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    
    '''
    
    VERSION = eval_version
    
    with open(test_dataset_path, 'r') as f:
        test_dataset = json.load(f)
    with open(orig_dev_path, 'r') as f:
        orig_dev_dataset = json.load(f)

    # len(rewriter_ILM_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

    ## Quick evaluation: only using the 1st ASR candidate

    # pred_idx = 0

#     ref_list = []
#     hyp_list = []
#     wer_numer = 0
#     wer_denom = 0

    for d in tqdm(test_dataset, desc=f'VERSION {VERSION}'):
        if len(d) == 0:
            continue

        c = d[0]

        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

        # Debug 
        # assert c['rewriter_tags'] == p['rewriter_tags'][:len(c['rewriter_tags'])], f"{c['rewriter_tags']}\n{p['rewriter_tags']}\nShould raise"

        _db_id = o['db_id']
        #_pred_sql = Question(_rewritten_question, _db_id, model_dict=model_dicts[rat_sql_run])[0]['inferred_code']
        
        _uskg_sample = get_uskg_sample(c)
        _uskg_sample['question'] = c['gold_question']
        _struct_in = get_uskg_struct_in(_uskg_sample)
        _pred_sql = play_pred("{}; structed knowledge: {}".format(_uskg_sample['question'], _struct_in),
                              model, tokenizer)[0]

        _gold_sql = c['query']
        ## verbose=False since too many get_sql() errors...
        _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id, verbose=False)

        # Save prediction results 
        c['pred_sql'] = _pred_sql
        c['score'] = _score
        c['exact'] = _exact
        c['exec'] = _exec

#         _question_toks = c['question_toks']
#         _question_toks = [_t.lower() for _t in _question_toks]
#         _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

#         ref_list.append([_gold_question_toks])
#         hyp_list.append(_question_toks)
#         wer_numer += editdistance.eval(_gold_question_toks, _question_toks)
#         wer_denom += len(_gold_question_toks)

    # Only using the 1st candidate to rewrite 
    _avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
    _avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
    _avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

    ## Std-dev (1st cand only)
    # _std_1st = np.std([d[0]['score'] for d in test_dataset])

    ## BLEU 
#     _bleu = corpus_bleu(list_of_references=ref_list,
#                         hypotheses=hyp_list)
#     _wer = 1.0 * wer_numer / (wer_denom + 1e-9)

    print('='*20, f'VERSION: {VERSION}', '='*20)
    print('avg_exact = {:.4f}'.format(_avg_exact_1st))
    # print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
    print('avg = {:.4f}'.format(_avg_1st))
    print('avg_exec = {:.4f}'.format(_avg_exec_1st))
    print(f'BLEU = {1:.4f}')
    print(f'WER = {0:.4f}')
    print('='*55)
    
    if test_output_path is not None:
        with open(test_output_path, 'w') as f:
            json.dump(test_dataset, f, indent=4)
    
    if result_output_path is not None:
        _res_d = {
            "avg_exact": _avg_exact_1st,
            "avg": _avg_1st,
            "avg_exec": _avg_exec_1st,
            "BLEU": 1.0,
            "WER": 0.0,
        }
        with open(result_output_path, 'w') as f:
            json.dump(_res_d, f, indent=4)


In [39]:
HUMAN_TEST = False
ASR = 'Amazon'

if not HUMAN_TEST:
    if ASR == 'AssemblyAI':
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    else:
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
else:
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

# Non ASR
if not HUMAN_TEST:
    if ASR == 'AssemblyAI':
        test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/uskg-test-save/Gold-text.json'
        result_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/uskg-test-save/eval-Gold-text.json'

    else:
        test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-test-save/Gold-text.json'
        result_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-test-save/eval-Gold-text.json'
else:
    test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-test-save/humantest-yshao-Gold-text.json'
    result_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-test-save/eval-humantest-yshao-Gold-text.json'

Full_evaluate_Gold_text(eval_version='Original (Gold-text)',
                       test_dataset_path=test_dataset_path,
                       orig_dev_path=orig_dev_path,
                       test_output_path=test_output_path,
                       result_output_path=result_output_path)


VERSION Original (Gold-text):   0%|          | 0/547 [00:00<?, ?it/s]

==================== VERSION: Original (Gold-text) ====================
avg_exact = 0.5832
avg = 0.6919
avg_exec = 0.6033
BLEU = 1.0000
WER = 0.0000


In [118]:
model_path

'/Users/mac/Desktop/syt/Deep-Learning/Repos/UnifiedSKG/output/server_runs/A-T5_base_prefix_spider_with_cell_value-asr_mixed/checkpoint-79500/'

In [119]:
## Test of uskg-mixed 

test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-mixed-test-save/Gold-text.json'
result_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/uskg-mixed-test-save/eval-Gold-text.json'

Full_evaluate_Gold_text(eval_version='Original (Gold-text)',
                       test_dataset_path=test_dataset_path,
                       orig_dev_path=orig_dev_path,
                       test_output_path=test_output_path,
                       result_output_path=result_output_path)


VERSION Original (Gold-text):   0%|          | 0/547 [00:00<?, ?it/s]

==================== VERSION: Original (Gold-text) ====================
avg_exact = 0.5393
avg = 0.6496
avg_exec = 0.5393
BLEU = 1.0000
WER = 0.0000


## Test of space and punct

### Harm of extra space

In [ ]:
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter+phonemes.json'
with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
len(test_dataset)

In [50]:
_tgt_sen = 'what are the different first names and ages of the students who do have pets?'
for i, d in enumerate(test_dataset):
    c = d[0]
    if c['question'] == _tgt_sen:
        print(i)

31


In [56]:
c = test_dataset[31][0]
_uskg_sample = get_uskg_sample(c)
_struct_in = get_uskg_struct_in(_uskg_sample)
print(_uskg_sample['question'], _struct_in)

what are the different first names and ages of the students who do have pets?  | pets_1 | student : stuid , lname , fname , age , sex , major , advisor , city_code | has_pet : stuid , petid | pets : petid , pettype , pet_age , weight


In [57]:
_pred_sql = play_pred("{}; structed knowledge: {}".format(_uskg_sample['question'], _struct_in),
                      model, tokenizer)[0]
_pred_sql

'select distinct t1.fname, t1.age from student as t1 join has_pet as t2 on t1.stuid = t2.stuid'

In [58]:
_text_in = _uskg_sample['question'][:-1] + ' ' + _uskg_sample['question'][-1]
_text_in

'what are the different first names and ages of the students who do have pets ?'

In [60]:
_pred_sql = play_pred("{}; structed knowledge: {}".format(_text_in, _struct_in),
                      model, tokenizer)[0]
_pred_sql

"select distinct t1.fname, t1.age from student as t1 join has_pet as t2 on t1.stuid = t2.stuid where pettype = 'pet'"

### Find other types of problematic samples

In [61]:
for i, d in enumerate(test_dataset):
    c = d[0]
    for t in c['question_toks']:
        if t[0] == "'":
            print(c['question'])

## Test of cell values

In [81]:
for i, d in enumerate(test_dataset):
    c = d[0]
    if 'PPT' in c['query']:
        print(i)

132
133
148
149


In [91]:
c = test_dataset[132][0]
c['question'], c['query']

('how many documents are using the template with type code PPT.',
 "SELECT count(*) FROM Documents AS T1 JOIN Templates AS T2 ON T1.Template_ID  =  T2.Template_ID WHERE T2.Template_Type_Code  =  'PPT'")

In [92]:
_uskg_sample = get_uskg_sample(c)
_struct_in = get_uskg_struct_in(_uskg_sample)
print(_struct_in)

 | cre_Doc_Template_Mgt | ref_template_types : template_type_code ( PPT ) , template_type_description | templates : template_id , version_number , template_type_code ( PPT ) , date_effective_from , date_effective_to , template_details | documents : document_id , template_id , document_name , document_description , other_details | paragraphs : paragraph_id , document_id , paragraph_text , other_details


In [93]:
_pred_sql = play_pred("{}; structed knowledge: {}".format(_uskg_sample['question'], _struct_in),
                      model, tokenizer)[0]
_pred_sql

'select count(*) from templates where template_type_code = "PPT"'

In [88]:
c = test_dataset[133][0]
c['question'], c['query']

('count the number of documents that use the P p T template type.',
 "SELECT count(*) FROM Documents AS T1 JOIN Templates AS T2 ON T1.Template_ID  =  T2.Template_ID WHERE T2.Template_Type_Code  =  'PPT'")

In [89]:
_uskg_sample = get_uskg_sample(c)
_struct_in = get_uskg_struct_in(_uskg_sample)
print(_struct_in)

 | cre_Doc_Template_Mgt | ref_template_types : template_type_code , template_type_description | templates : template_id , version_number , template_type_code , date_effective_from , date_effective_to , template_details | documents : document_id , template_id , document_name , document_description , other_details | paragraphs : paragraph_id , document_id , paragraph_text , other_details


In [90]:
_pred_sql = play_pred("{}; structed knowledge: {}".format(_uskg_sample['question'], _struct_in),
                      model, tokenizer)[0]
_pred_sql

'select count(*) from documents where template_type_code = "P p t"'

# Create sample for figure 1

In [97]:
for d in test_dataset:
    c = d[0]
    if c['db_id'] == 'flight_2':
        print(c['question'])
#         _uskg_sample = get_uskg_sample(c)
#         _struct_in = get_uskg_struct_in(_uskg_sample)
#         print(_struct_in)
        break

which country does airline JetBlue Airways belong to?


In [109]:
c = deepcopy(c)
c['question'] = 'How many flights go from NY to CA?'
# c['question'] = 'How many flights go from and why to see a?'
# c['question_toks'] = nltk.word_tokenize(c['question'])
_uskg_sample = get_uskg_sample(c)
_struct_in = get_uskg_struct_in(_uskg_sample)
print(_struct_in)

 | flight_2 | airlines : uid , airline , abbreviation , country | airports : city , airportcode , airportname , country , countryabbrev | flights : airline , flightno , sourceairport , destairport


In [110]:
_text_in = spider.normalize(_uskg_sample["question"])
_text_in

'how many flights go from ny to ca?'

In [106]:
play("{}; structed knowledge: {}".format(_text_in, _struct_in), model, tokenizer)

=====❓Request=====
how many flights go from and why to see a?; structed knowledge:  | flight_2 | airlines : uid , airline , abbreviation , country | airports : city , airportcode , airportname , country , countryabbrev | flights : airline , flightno , sourceairport , destairport
=====💡Answer=====
['select sourceairport, count(*) from flights group by sourceairport']


In [111]:
play("{}; structed knowledge: {}".format(_text_in, _struct_in), model, tokenizer)

=====❓Request=====
how many flights go from ny to ca?; structed knowledge:  | flight_2 | airlines : uid , airline , abbreviation , country | airports : city , airportcode , airportname , country , countryabbrev | flights : airline , flightno , sourceairport , destairport
=====💡Answer=====
['select count(*) from flights where sourceairport = "NY" and sourceairport = "ca"']


# Temp

In [74]:
_s = "I 've got `` a '' , b , and c with 1.0 and 100,000 .".split(' ')
print(_s)

['I', "'ve", 'got', '``', 'a', "''", ',', 'b', ',', 'and', 'c', 'with', '1.0', 'and', '100,000', '.']


In [72]:
_detokenizer = TreebankWordDetokenizer()
_detokenizer.detokenize(_s)

'I\'ve got "a", b, and c with 1.0 and 100,000.'

In [95]:
len(test_dataset)

547